In [2]:
testdata = """class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12""".splitlines()

In [3]:
def parse_data(data):
    fields = dict()
    my_ticket = []
    nearby_tickets = []

    section = 1
    for line in data:
        if line == '':
            section += 1
            continue

        if line[-1] == ':':
            continue

        if section == 1:
            field, ranges = line.split(':')
            range1, range2 = ranges.split(' or ')
            range1_low, range1_high = range1.split('-')
            range2_low, range2_high = range2.split('-')

            fields[field] = [(int(range1_low), int(range1_high)), (int(range2_low), int(range2_high))]
        elif section == 2:
            my_ticket = [int(p) for p in line.split(',')]
        elif section == 3:
            nearby_tickets.append([int(p) for p in line.split(',')])

    return fields, my_ticket, nearby_tickets

In [4]:
parse_data(testdata)

({'class': [(1, 3), (5, 7)],
  'row': [(6, 11), (33, 44)],
  'seat': [(13, 40), (45, 50)]},
 [7, 1, 14],
 [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [9]:
def validate(ticket, fields):
    error = 0
    for parameter in ticket:
        if not validate_parameter(parameter, fields):
            error += parameter
    return error

def validate_parameter(parameter, fields):
    for field, ranges in fields.items():    
        if (ranges[0][0] <= parameter <= ranges[0][1] or ranges[1][0] <= parameter <= ranges[1][1]):
            # print(f'parameter {parameter} match field {field} range {ranges}')
            return True
    print(f'no field matched parameter {parameter}')
    return False

In [10]:
fields, my_ticket, nearby_tickets = parse_data(testdata)
print(fields, my_ticket, nearby_tickets)
print(sum([validate(ticket, fields) for ticket in nearby_tickets]))

{'class': [(1, 3), (5, 7)], 'row': [(6, 11), (33, 44)], 'seat': [(13, 40), (45, 50)]} [7, 1, 14] [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]]
no field matched parameter 4
no field matched parameter 55
no field matched parameter 12
71


In [11]:
with open('input', 'r') as inp:
   inputdata = [line.strip() for line in inp.readlines()]

fields, my_ticket, nearby_tickets = parse_data(inputdata)
#print(fields, my_ticket, nearby_tickets)
print(sum([validate(ticket, fields) for ticket in nearby_tickets]))

no field matched parameter 994
no field matched parameter 14
no field matched parameter 982
no field matched parameter 998
no field matched parameter 3
no field matched parameter 4
no field matched parameter 981
no field matched parameter 977
no field matched parameter 23
no field matched parameter 995
no field matched parameter 3
no field matched parameter 8
no field matched parameter 11
no field matched parameter 2
no field matched parameter 997
no field matched parameter 998
no field matched parameter 997
no field matched parameter 985
no field matched parameter 985
no field matched parameter 983
no field matched parameter 3
no field matched parameter 975
no field matched parameter 20
no field matched parameter 12
no field matched parameter 978
no field matched parameter 979
no field matched parameter 2
no field matched parameter 975
no field matched parameter 983
no field matched parameter 982
no field matched parameter 986
no field matched parameter 975
no field matched parameter 